In [ ]:
#Code to mount Google Drive at Colab Notebook instance
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##**Install Langchain and Google Gemini related libraries**

In [ ]:
!pip install -q -U langchain
!pip install -q -U google-generativeai langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 2.2 MB/s eta 0:00:00


# Get Gemini API Key from Secrets
Set GEMINI_KEY secret key at Google Colab and get that here to runn Google Gemini LLM. You can get Google Gemini Key from following link https://makersuite.google.com/app/apikey

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GEMINI_KEY')

Copy chinhook.db to your Google Drive

Now, chinhook.db is in our directory and we can interface with it using the SQLAlchemy-driven SQLDatabase class.

Location of chinhook.db
/content/drive/MyDrive/Colab Notebooks/chinook.db

In [ ]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:////content/drive/MyDrive/Colab Notebooks/chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artists LIMIT 10;")

sqlite
['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'tracks']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

# Chain
Let’s create a simple chain that takes a question, turns it into a SQL query, executes the query, and uses the result to answer the original question.

# Convert question to SQL query
The first step in a SQL chain or agent is to take the user input and convert it to a SQL query. LangChain comes with a built-in chain for this: create_sql_query_chain

In [ ]:
from langchain.chains import create_sql_query_chain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY, convert_system_message_to_human=True, temperature=0.0)
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there"})
response

'SQLQuery: SELECT COUNT(*) FROM employees;'

In [ ]:
db.run(response)

'[(8,)]'

#Execute SQL query
Now that we’ve generated a SQL query, we’ll want to execute it.

We can use the QuerySQLDatabaseTool to easily add query execution to our chain:

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)

write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
chain.invoke({"question": "How many employees are there"})

'Error: (sqlite3.OperationalError) near "SQLQuery": syntax error\n[SQL: SQLQuery: SELECT COUNT(*) FROM employees;]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)'

In [ ]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.prompts import PromptTemplate

execute_query = QuerySQLDataBaseTool(db=db)

template = '''Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the {top_k} answer.
Use the following format:

Question: "Question here"
"SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}.

Question: {input}'''
prompt = PromptTemplate.from_template(template)

write_query = create_sql_query_chain(llm, db, prompt)
chain = write_query | execute_query
chain.invoke({"question": "How many employees are there"})

'[(8,)]'

#Answer the question
Now that we’ve got a way to automatically generate and execute queries, we just need to combine the original question and SQL query result to generate a final answer.

In [ ]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
{query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many employees are there"})

'There are 8 employees.'

# Agents
LangChain has an SQL Agent which provides a more flexible way of interacting with SQL databases. The main advantages of using the SQL Agent are:

It can answer questions based on the databases’ schema as well as on the databases’ content (like describing a specific table).
It can recover from errors by running a generated query, catching the traceback and regenerating it correctly.
It can answer questions that require multiple dependent queries.
It will save tokens by only considering the schema from relevant tables.
To initialize the agent, we use create_sql_agent function. This agent contains the SQLDatabaseToolkit which contains tools to:

- Create and execute queries
- Check query syntax
- Retrieve table descriptions

In [ ]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents.agent_types import AgentType
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

agent_executor = create_sql_agent(llm,
                                  toolkit=SQLDatabaseToolkit(db=db, llm=llm),
                                  agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                                  verbose=True)

In [ ]:
agent_executor.invoke(
    {
        "input": "List the total sales per country. Which country's customers spent the most?"
    }
)



> Entering new AgentExecutor chain...
I should first list all the countries and their total sales. Then I can find the country with the highest total sales.
Action: sql_db_list_tables
Action Input: albums, artists, customers, employees, genres, invoice_items, invoices, media_types, playlist_track, playlists, tracksAction: sql_db_query_checker
Action Input: SELECT Country, SUM(Total) AS TotalSales FROM Invoices GROUP BY Country ORDER BY TotalSales DESCSELECT Country, SUM(Total) AS TotalSales 
FROM Invoices 
GROUP BY Country 
ORDER BY TotalSales DESCAction: sql_db_query
Action Input: SELECT Country, SUM(Total) AS TotalSales FROM Invoices GROUP BY Country ORDER BY TotalSales DESCError: (sqlite3.OperationalError) no such column: Country
[SQL: SELECT Country, SUM(Total) AS TotalSales FROM Invoices GROUP BY Country ORDER BY TotalSales DESC]
(Background on this error at: https://sqlalche.me/e/20/e3q8)Action: sql_db_schema
Action Input: InvoicesError: table_names {'Invoices'} not found in da

{'input': "List the total sales per country. Which country's customers spent the most?",
 'output': "The USA's customers spent the most money, with a total of 523.06."}